In [ ]:
#Running PIFuHD - https://shunsukesaito.github.io/PIFuHD/
'''
To make it cross compatible with RigNet - https://zhan-xu.github.io/rig-net/

    conda create -n <env_name> python=3.7
    conda activate <env_name>
    conda install pytorch==1.6.0 torchvision==0.7.0 cudatoolkit=10.1 -c pytorch
    pip install numpy scipy matplotlib tensorboard open3d==0.9.0 opencv-python

Requirements - pip install -r requirements.txt

    Python 3
    PyTorch tested on 1.4.0
    json
    PIL
    skimage
    tqdm
    numpy
    cv2
'''

In [ ]:
ls

In [ ]:
cd pifuHD\pifuhd\sample_images

In [ ]:
%cd ..
%cd lightweight-human-pose-estimation.pytorch

In [ ]:
!pip install -U scikit-image
!pip install -U cython 
!pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

In [2]:
pwd

'D:\\CG_Source\\NeRFs\\pifuHD\\pifuhd'

In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [1]:
%cd pifuHD/pifuhd/


D:\CG_Source\NeRFs\pifuHD\pifuhd


In [2]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.


from apps.recon import reconWrapper
import argparse


###############################################################################################
##                   Setting
###############################################################################################
'''parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input_path', type=str, default='./sample_images')
parser.add_argument('-o', '--out_path', type=str, default='./results')
parser.add_argument('-c', '--ckpt_path', type=str, default='./checkpoints/pifuhd.pt')
parser.add_argument('-r', '--resolution', type=int, default=512)
parser.add_argument('--use_rect', action='store_true', help='use rectangle for cropping')
args = parser.parse_args()'''
###############################################################################################
##                   Upper PIFu
###############################################################################################

#resolution = str(args.resolution)

start_id = -1
end_id = -1
cmd = ['--dataroot', './sample_images', '--results_path', './results',\
       '--loadSize', '1024', '--resolution', str(128), '--load_netMR_checkpoint_path', \
       './checkpoints/pifuhd.pt',\
       '--start_id', '%d' % start_id, '--end_id', '%d' % end_id]
reconWrapper(cmd, 'store_true')

Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal


  0%|                                                                                          | 0/1 [00:00<?, ?it/s]

initialize network with normal
generate mesh (test) ...
./results/pifuhd_final/recon/result_IMG_3392.2_128.obj


  0%|                                                                                          | 0/1 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 6.00 GiB total capacity; 4.05 GiB already allocated; 63.62 MiB free; 4.26 GiB reserved in total by PyTorch)

In [3]:
#checkpoints downloaded from - https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
!python -m apps.simple_test

Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
./results/pifuhd_final/recon/result_test_512.obj


  0%|          | 0/1 [00:04<?, ?it/s]
Traceback (most recent call last):
  File "C:\miniconda3\envs\3d_avatar\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\miniconda3\envs\3d_avatar\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\CG_Source\NeRFs\pifuHD\pifuhd\apps\simple_test.py", line 30, in <module>
    reconWrapper(cmd, args.use_rect)
  File "D:\CG_Source\NeRFs\pifuHD\pifuhd\apps\recon.py", line 220, in reconWrapper
    recon(opt, use_rect)
  File "D:\CG_Source\NeRFs\pifuHD\pifuhd\apps\recon.py", line 210, in recon
    gen_mesh(opt.resolution, netMR, cuda, test_data, save_path, components=opt.use_compose)
  File "D:\CG_Source\NeRFs\pifuHD\pifuhd\apps\recon.py", line 39, in gen_mesh
    net.filter_local(image_tensor[:,None])
  File "D:\CG_Source\NeRFs\pifuHD\pifuhd\lib\model\HGPIFuMRNet.py", line 117, in filter_local
    self.im_feat_list, self.normx = self.image_filter(images.view(-1,*images.size()[2:]))
  File "C:\

In [4]:
#clearing memory in GPU
import torch
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
from platform import python_version
print(python_version())